In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import json
from selenium.webdriver.chrome.options import Options
from bs4 import UnicodeDammit
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import lxml.html as lh

options = Options()
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.517 Safari/537.36'
options.add_argument('user-agent={0}'.format(user_agent))

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=options)
driver.get("https://www.oddschecker.com/horse-racing/2022-11-21-kempton/13:50/winner")
html_source_code = driver.execute_script("return document.body.innerHTML;")
html_soup: BeautifulSoup = BeautifulSoup(html_source_code, 'html.parser')
prettyHTML = html_soup.prettify()
driver.close()

In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import json
from selenium.webdriver.chrome.options import Options
from bs4 import UnicodeDammit
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import lxml.html as lh

In [11]:
#divide array into sub array for each horse, then 
#["data-o"]
#[data-ew-denom"]
#["data-ew-places"]
class OddsCheckerPrice:
    
    def __init__(self,bookmaker = None, name = None, price = None, ewDenom = None, ewPlaces =  None):
        self.name = name
        self.bookie = bookmaker
        self.price = price
        self.ewDenominator = ewDenom
        self.ewPlaces = ewPlaces
        
    def printPrice(self):
        print("Name: ", self.name,
              " Bookmaker: ", self.bookie,
              " Price: ", self.price,
              " ewDenom: ", self.ewDenominator,
              " ewPlaces: ", self.ewPlaces)

In [12]:
def oddsPortalScraper(prettyHTML, bookMakerList, url, log):
    
    finalPrices = []
    
    def split(list_a, chunk_size):
        for i in range(0, len(list_a), chunk_size):
            yield list_a[i:i + chunk_size]
    
    soup = BeautifulSoup(prettyHTML)
    body = soup.body
    pricesHtml = body.find_all("td", class_= lambda value: value and 
                     (value.startswith("bc bs") or value.startswith("np o") or value.startswith("o np")))
    
    if log == True:
        f = open(url + ".txt", "wb")
        f.write(str(body).encode("utf-8"))
        f.close()
    
    pricesHtml = list(split(pricesHtml, len(bookMakerList)))
    
    namesHtml = body.find_all("a", class_ = "popup selTxt")
    
    horseNames = [x["data-name"] for x in namesHtml]
    
    for i in range(0, len(pricesHtml)):
        horsePricesHtml = pricesHtml[i]
        
        for b in range(0, len(bookMakerList)):
            priceHtml = horsePricesHtml[b]
            
            if priceHtml["data-o"] == "" or priceHtml.has_attr("data-ew-denom") == False:
                continue
            else:
                price = priceHtml["data-o"]
                ewDenom = priceHtml["data-ew-denom"]
                ewPlaces = priceHtml["data-ew-places"]
                bookMakerName = bookMakerList[b]
                
                finalPrices.append(OddsCheckerPrice(bookMakerName,horseNames[i], price, ewDenom, ewPlaces))
                
    return finalPrices

In [13]:
def scrapeOddsCheckerRace(url, bookmakers, log = False):
    
    options = Options()
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.517 Safari/537.36'
    options.add_argument('user-agent={0}'.format(user_agent))

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=options)
    driver.get(url)
    html_source_code = driver.execute_script("return document.body.innerHTML;")
    html_soup: BeautifulSoup = BeautifulSoup(html_source_code, 'html.parser')
    prettyHTML = html_soup.prettify()
    driver.close()
    
    prices = oddsPortalScraper(prettyHTML, bookmakers,url, log)
    
    return prices
  

In [33]:
import httplib2  
def scrapeOddsChecker(urlList, bookmakers, log = False):
    prices = []
    for url in urlList:
        #need to add some validation to check whether this url exists, currenty not able to do so..
        racePrices = scrapeOddsCheckerRace(url, bookmakers, log)
        prices.append(racePrices)
        
    return prices

In [34]:
bookmakers = ["Bet365", "SkyBet", "PaddyPower", "WilliamHill", "888Sport",
             "Betfair", "BetVictor", "Coral", "UniBet", "SpreadEx", "BetFred",
             "BoyleSports", "10Bet", "BetUk", "SportingIndex", "LiveScoreBet",
             "QuinnBet", "BetWay", "LadBrokes", "PariMatch", "VBet", "SBK", "Tote",
             "BetFairExchange", "Smarkets", "MatchBook"]

urlList = ["https://www.oddschecker.com/horse-racing/2022-11-21-kempton/13:50/winner",
           "https://www.oddschecker.com/horse-racing/2022-11-21-wagga/03:25/winner"]

prices = scrapeOddsChecker(urlList, bookmakers, False)
#prices = scrapeOddsCheckerRace("https://www.oddschecker.com/horse-racing/2022-11-21-kempton/13:50/winner", bookmakers, False)